# 十二、训练深度神经网络

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd

### [Batch Normalization](https://zhuanlan.zhihu.com/p/38176412)

尽管使用 He 初始化和 ELU（或任何 ReLU 变体）可以显著减少训练开始阶段的梯度消失/爆炸问题，但不能保证在训练期间问题不会再次出现。

[在 2015 年的一篇论文中](https://links.jianshu.com/go?to=https%3A%2F%2Farxiv.org%2Fabs%2F1502.03167)，Sergey Ioffe 和 Christian Szegedy 提出了一种称为批归一化（Batch Normalization，BN）的方法来解决梯度消失/爆炸问题。该方法包括在每层的激活函数之前或之后在模型中添加操作。操作就是将输入平均值变为 0，方差变为 1，然后用两个新参数，一个做缩放，一个做偏移。换句话说，这个操作可以让模型学习到每层输入值的最佳缩放值和平均值。大大多数情况下，如果模型的第一层使用了 BN 层，则不用标准化训练集（比如使用`StandardScaler`）；BN 层做了标准化工作（虽然是近似的，每次每次只处理一个批次，但能做缩放和平移）。

为了对输入进行零居中（平均值是 0）和归一化，算法需要估计输入的均值和标准差。 它通过评估当前小批量输入的均值和标准差（因此命名为“批归一化”）来实现。 整个操作在公式中。


<div align=center><img width="500" height="300" src="./static/1.jpg"/></div>


#### 注意：

你可能会发现，训练相当缓慢，这是因为每个周期都因为使用 BN 而延长了时间。但是有了 BN，收敛的速度更快，需要的周期数更少。综合来看，需要的总时长变短了。

### 使用 Keras 实现批归一化

In [5]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300,activation='elu',kernel_initializer='he_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100,activation='elu',kernel_initializer='he_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10,activation='softmax')
])

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 784)               3136      
_________________________________________________________________
dense_2 (Dense)              (None, 300)               235500    
_________________________________________________________________
batch_normalization_3 (Batch (None, 300)               1200      
_________________________________________________________________
dense_3 (Dense)              (None, 100)               30100     
_________________________________________________________________
batch_normalization_4 (Batch (None, 100)               400       
_________________________________________________________________
dense_4 (Dense)              (None, 10)                1

可以看到每个 BN 层添加了四个参数：γ、 β、 μ 和 σ（例如，第一个 BN 层添加了 3136 个参数，即4 × 784）。后两个参数μ 和 σ是移动平均，不受反向传播影响，Keras 称其“不可训练”（如果将 BN 的总参数3,136 + 1,200 + 400除以 2，得到 2368，就是模型中总的不可训练的参数量）。

看下第一个 BN 层的参数。两个参数是可训练的（通过反向传播），两个不可训练：

In [7]:
[(var.name ,var.trainable) for var in model.layers[1].variables]

[('batch_normalization_2/gamma:0', True),
 ('batch_normalization_2/beta:0', True),
 ('batch_normalization_2/moving_mean:0', False),
 ('batch_normalization_2/moving_variance:0', False)]

当在 Keras 中创建一个 BN 层时，训练过程中，还会创建两个 Keras 在迭代时的操作。该操作会更新移动平均值。因为后端使用的是 TensorFlow，这些操作就是 TensorFlow 操作（第 12 章会讨论 TF 操作）：

In [12]:
model.layers[1].updates

D:\APYPLA~1\venv_dir\venv1\lib\site-packages\keras\engine\base_layer.py:1348: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`layer.updates` will be removed in a future version. '


[]

BN 的论文作者建议在激活函数之前使用 BN 层，而不是像前面的例子添加到后面。到底是前面还是后面好存在争议，取决于具体的任务 —— 你最好在数据集上试验一下哪种选择好。要在激活函数前添加 BN 层，必须将激活函数从隐藏层拿出来，单独做成一层。另外，因为 BN 层对每个输入有一个偏移参数，可以将前一层的偏置项去掉（设置use_bias=False）：

In [57]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[6]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300,kernel_initializer='he_normal',use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Dense(100,kernel_initializer='he_normal',use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Dense(10,activation='softmax')
])

### 梯度裁剪

减少梯度爆炸问题的一种常用技术是在反向传播过程中剪切梯度，使它们不超过某个阈值，这种方法称为梯度裁剪。梯度裁剪在循环神经网络中用的很多，因为循环神经网络中用 BN 很麻烦，参见第 15 章。 对于其它类型的网络，BN 就足够了。在 Keras 中，梯度裁剪只需在创建优化器时设置clipvalue或clipnorm参数，如下：

In [58]:
optimizer = keras.optimizers.SGD(clipvalue=1.0)
model.compile(loss='mse',optimizer=optimizer)

优化器会将梯度向量中的每个值裁剪到 -1.0 和 1.0 之间。这意味着损失（对每个可训练参数）的所有偏导数会被裁剪到 -1.0 和 1.0 之间。阈值是一个可以调节的超参数，可能影响到梯度向量的方向。例如，如果原始梯度向量是 `[0.9, 100.0]` ，它大体指向第二个轴；但在裁剪之后变为 `[0.9, 1.0]` ，方向就大体指向对角线了。在实际中，梯度裁剪的效果不错。如果想确保梯度裁剪不改变梯度向量的方向，就需要设置clipnorm靠范数裁剪，这样如果梯度的 l2 范数超过了阈值，就能对整个梯度裁剪。例如，如果设置 `clipnorm = 1.0` ，向量 `[0.9, 100.0]` 就会被裁剪为 `[0.00899964, 0.9999595]` ，方向没变，但第一个量几乎被抹去了。如果再训练过程中发现了梯度爆炸（可以用 TensorBoard 跟踪梯度），最好的方法是既用值也用范数裁剪，设置不同的阈值，看看哪个在验证集上表现最好。

### 复用预训练层

从零开始训练一个非常大的 DNN 通常不是一个好主意，相反，您应该总是尝试找到一个现有的神经网络来完成与您正在尝试解决的任务类似的任务（第 14 章会介绍如何找），然后复用这个网络的较低层：这就是所谓的迁移学习。这样不仅能大大加快训练速度，还将需要更少的训练数据。

例如，假设你有一个经过训练的 DNN，能将图片分为 100 个不同的类别，包括动物，植物，车辆和日常物品。 现在想要训练一个 DNN 来对特定类型的车辆进行分类。 这些任务非常相似，甚至部分重叠，因此应该尝试重新使用第一个网络的一部分（请参见图 ）。

<div align=center><img width="500" height="300" src="./static/2.jpg"/></div>


#### 笔记：
如果新任务的输入图像与原始任务中使用的输入图像的大小不一致，则必须添加预处理步骤以将其大小调整为原始模型的预期大小。 更一般地说，如果输入具有类似的低级层次的特征，则迁移学习将很好地工作。

原始模型的输出层通常要替换掉，因为对于新任务可能一点用也没有，输出的数量可能就不对。相似的，原始模型的上层也不如浅层管用，因为高阶特征可能相差很大。需要确定好到底用几层。

#### 提示：

任务越相似，可复用的层越多。对于非常相似的任务，可以尝试保留所有的隐藏层，替换输出层。

先将所有复用的层冻结（即，使其权重不可训练，梯度下降不能修改权重），然后训练模型，看其表现如何。然后将复用的最上一或两层解冻，让反向传播可以调节它们，再查看性能有无提升。训练数据越多，可以解冻的层越多。解冻时减小学习率也有帮助，可以避免破坏微调而得的权重。

如果效果不好，或者训练数据不多，可以尝试去除顶层，将其余的层都解冻。不断尝试，直到找到合适的层，如果训练数据很多，可以尝试替换顶层，或者加入更多的隐藏层。

### 用 Keras 进行迁移学习

看一个例子。假设 Fashion MNIST 只有八个类，不包括拖鞋和 T 恤。一些人在这个数据集上搭建并训练了一个 Keras 模型，且效果不错（准确率大于 90%），将其称为模型 A。现在想处理另一个问题：有拖鞋和 T 恤的图片，要训练一个二分类器（`positive=shirt, negative=sandal`）。数据集不大，只有 200 张打了标签的图片。当训练架构与模型 A 相同的新模型时（称其为模型 B），表现非常好（准确率 97.2%）。但因为这是一个非常简单的任务（只有两类），所以准确率应该还可以更高。因为和任务 A 很像，所以可以尝试一下迁移学习。

In [27]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [28]:
hoursing = fetch_california_housing()

x_train_full,x_test,y_train_full,y_test = train_test_split(hoursing.data,hoursing.target)
x_train,x_valid,y_train,y_valid = train_test_split(x_train_full,y_train_full) 

In [29]:
ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_valid = ss.transform(x_valid)
x_test = ss.transform(x_test)

In [45]:
x_train_A,x_train_B = x_train[:,:5],x_train[:,2:]
x_valid_A,x_valid_B = x_valid[:,:5],x_valid[:,2:]
x_test_A,x_test_B = x_test[:,:5],x_test[:,2:]
x_new_A,x_new_B = x_test_A[:3],x_test_B[:3]

In [37]:
x_train_B.shape

(11610, 6)

首先，加载模型 A，创建一个新模型，除了输出层不要，保留所有的层：

In [17]:
# model_A = keras.models.load_model('my_keras_model.h5')
# model_A_on_B = keras.models.Sequential(model_A.layers[:-1])
# model_A_on_B.add(keras.layers.Dense(1,activation='sigmoid'))

model_A 和 model_B_on_A 公用了一些层。当你训练model_B_on_A时，也会影响model_A。如果想避免，需要在复用前克隆model_A。要这么做，可以使用clone.model()，然后复制权重（clone.model()不能克隆权重）：

In [59]:
model_A_clone = keras.models.clone_model(model)
model_A_clone.set_weights(model.get_weights()) 

现在就可以训练model_A了，但是因为新输出层是随机初始化的，误差较大，较大的误差梯度可能会破坏复用的权重。为了避免，一种方法是在前几次周期中，冻结复用的层，让新层有时间学到合理的权重。要实现的话，将每层的trainable属性设为False，然后编译模型：

In [60]:
for layer in model_A_clone.layers[:-1]:
    layer.trainable =False

model_A_clone.compile(loss='mse',optimizer='sgd',metrics=['accuracy'])

#### 笔记：

冻结或解冻模型之后，都需要编译.

训练几个周期之后，就可以解冻复用层（需要再次编译模型），然后接着训练以微调模型。解冻之后，最好降低学习率，目的还是避免破坏复用层的权重：

In [61]:
history = model_A_clone.fit(x_train_B,y_train,epochs=4,validation_data=(x_valid_B,y_valid))

Epoch 1/4
363/363 [==============================] - 1s 2ms/step - loss: 5.2240 - accuracy: 0.0012 - val_loss: 5.1178 - val_accuracy: 7.7519e-04
Epoch 2/4
363/363 [==============================] - 1s 1ms/step - loss: 5.2218 - accuracy: 0.0011 - val_loss: 5.1166 - val_accuracy: 7.7519e-04
Epoch 3/4
363/363 [==============================] - 0s 1ms/step - loss: 5.2209 - accuracy: 0.0013 - val_loss: 5.1160 - val_accuracy: 7.7519e-04
Epoch 4/4
363/363 [==============================] - 0s 1ms/step - loss: 5.2204 - accuracy: 0.0013 - val_loss: 5.1155 - val_accuracy: 7.7519e-04


In [56]:
for layer in model_A_clone.layers[:-1]:
    layer.trainable = True

optimizer = keras.optimizers.SGD(lr=1e-4) # the default lr is 1e-2
model_A_clone.compile(loss="mse", optimizer=optimizer,
                     metrics=["accuracy"])
history = model_A_clone.fit(x_train_B, y_train, epochs=16,
                           validation_data=(x_valid_B, y_valid)) 

D:\APYPLA~1\venv_dir\venv1\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/16
363/363 [==============================] - 2s 2ms/step - loss: 5.2230 - accuracy: 6.8906e-04 - val_loss: 5.1151 - val_accuracy: 0.0016
Epoch 2/16
363/363 [==============================] - 1s 2ms/step - loss: 5.2230 - accuracy: 9.4746e-04 - val_loss: 5.1151 - val_accuracy: 0.0010
Epoch 3/16
363/363 [==============================] - 1s 2ms/step - loss: 5.2230 - accuracy: 6.0293e-04 - val_loss: 5.1155 - val_accuracy: 0.0013
Epoch 4/16
363/363 [==============================] - 1s 2ms/step - loss: 5.2230 - accuracy: 8.6133e-04 - val_loss: 5.1153 - val_accuracy: 0.0013
Epoch 5/16
363/363 [==============================] - 1s 2ms/step - loss: 5.2229 - accuracy: 6.8906e-04 - val_loss: 5.1156 - val_accuracy: 0.0010
Epoch 6/16
363/363 [==============================] - 1s 2ms/step - loss: 5.2229 - accuracy: 6.0293e-04 - val_loss: 5.1151 - val_accuracy: 0.0013
Epoch 7/16
363/363 [==============================] - 1s 2ms/step - loss: 5.2229 - accuracy: 6.8906e-04 - val_loss: 5.1153 -

In [63]:
model_A_clone.evaluate(x_test_B, y_test) # 前面是误差率，后面的是准确率

162/162 [==============================] - 0s 1ms/step - loss: 5.2551 - accuracy: 9.6899e-04


[5.255092620849609, 0.0009689922444522381]

### [更快的优化器](https://hands1ml.apachecn.org/#/docs/11?id=%e6%9b%b4%e5%bf%ab%e7%9a%84%e4%bc%98%e5%8c%96%e5%99%a8)

训练一个非常大的深度神经网络可能会非常缓慢。 到目前为止，我们已经看到了四种加速训练的方法（并且达到更好性能的方法）：对连接权重应用良好的初始化策略，使用良好的激活函数，使用批归一化以及重用预训练网络的部分（使用辅助任务或无监督学习）。 另一个速度提升的方法是使用更快的优化器，而不是常规的梯度下降优化器。 在本节中，我们将介绍最流行的算法：动量优化，Nesterov 加速梯度，AdaGrad，RMSProp，最后是 Adam 和 Nadam 优化。

#### 剧透：

本节的结论是，几乎总是应该使用 `Adam_optimization` ，所以如果不关心它是如何工作的，只需使用 `AdamOptimizer` 替换 `GradientDescentOptimizer` ，然后跳到下一节！ 只需要这么小的改动，训练通常会快几倍。 但是，`Adam 优化` 确实有三个可以调整的超参数（加上学习率）。 默认值通常工作的不错，但如果您需要调整它们，知道他们怎么实现的可能会有帮助。 `Adam 优化` 结合了来自其他优化算法的几个想法，所以先看看这些算法是有用的。

### ℓ1 和 ℓ2 正则

就像第 4 章中对简单线性模型所做的那样，可以使用 ℓ2 正则约束一个神经网络的连接权重，或 ℓ1 正则得到稀疏模型（许多权重为 0）。下面是对 Keras 的连接权重设置 ℓ2 正则，正则因子是 0.01：

l2函数返回的正则器会在训练中的每步被调用，以计算正则损失。正则损失随后被添加到最终损失。如果要使用 ℓ1 正则，可以使用keras.regularizers.l1()；如果想使用 ℓ1 和 ℓ2 正则，可以使用keras.regularizers.l1_l2()（要设置两个正则因子）。

因为想对模型中的所有层使用相同的正则器，还要使用相同的激活函数和相同的初始化策略。参数重复使代码很难看。为了好看，可以用循环重构代码。另一种方法是使用 Python 的函数functools.partial()，它可以为任意可调回对象创建封装类，并有默认参数值：

In [64]:
from functools import partial

In [65]:
RegularizedDense = partial(keras.layers.Dense,
                           activation='elu',
                           kernel_initializer='he_normal',
                           kernel_regularizer = keras.regularizers.l2(0.01))

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    RegularizedDense(300),
    RegularizedDense(100),
    RegularizedDense(10,activation='softmax',kernel_initializer='glorot_uniform'),
])

### [Dropout层](https://github.com/OneStepAndTwoSteps/Data_Analysis_notes/tree/master/2%E3%80%81%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90%E3%80%81%E6%9C%BA%E5%99%A8%E5%AD%A6%E4%B9%A0/%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90%E7%AE%97%E6%B3%95/%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C/Dropout) 

有一个小而重要的技术细节。 假设 `p = 50%`，在这种情况下，`在测试期间`，在训练期间神经元将被连接到两倍于（平均）的输入神经元。 为了弥补这个事实，我们需要在训练之后将每个神经元的输入连接权重乘以 `0.5`。 如果我们不这样做，每个神经元的总输入信号大概是网络训练的两倍，这不太可能表现良好。 更一般地说，我们需要将每个输入连接权重乘以训练后的保持概率`（1-p）`。 或者，我们可以在训练过程中将每个神经元的输出除以保持概率（这些替代方案并不完全等价，但它们工作得同样好）。

要使用 Kera 实现丢弃，可以使用keras.layers.Dropout层。在训练过程中，它随机丢弃一些输入（将它们设置为 0），并用保留概率来划分剩余输入。 训练结束后，这个函数什么都不做，只是将输入传给下一层。下面的代码将丢弃正则化应用于每个紧密层之前，丢弃率为 0.2：

In [67]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.Dropout(rate = 0.2),
    keras.layers.Dense(300,activation='elu',kernel_initializer='he_normal'),
    keras.layers.Dropout(rate = 0.2),
    keras.layers.Dense(100,activation='elu',kernel_initializer='he_normal'),
    keras.layers.Dropout(rate = 0.2),
    keras.layers.Dense(10,activation='softmax'),

])

#### 警告：

因为丢弃只在训练时有用，比较训练损失和验证损失会产生误导。特别地，一个模型可能过拟合训练集，但训练和验证损失相近。因此一定要不要带丢弃评估训练损失（比如训练后）。

如果观察到模型 `过拟合`，则可以 `增加丢弃率`（即，减少 `keep_prob` 超参数）。 相反，如果模型 `欠拟合` 训练集，则应尝试 `降低丢弃率`（即增加`keep_prob`）。 它也可以帮助增加大层的丢弃率，并减少小层的丢弃率。另外，许多优秀的架构只在最后一个隐藏层之后使用丢弃，如果全都加上丢弃太强了，可以这么试试。

丢弃似乎减缓了收敛速度，但通常会在调参得当时使模型更好。 所以，这通常值得花费额外的时间和精力。

#### 提示：

如果想对一个自归一化的基于 SELU 的网络使用正则，应该使用 alpha 丢弃：这是一个丢弃的变体，可以保留输入的平均值和标准差（它是在 SELU 的论文中提出的，因为常规的丢弃会破会自归一化）。

### [蒙特卡洛（MC）丢弃](https://hands1ml.apachecn.org/#/docs/11?id=%e8%92%99%e7%89%b9%e5%8d%a1%e6%b4%9b%ef%bc%88mc%ef%bc%89%e4%b8%a2%e5%bc%83)

In [69]:
# y_probas = np.stack([model(X_test_scaled, training=True)
#                      for sample in range(100)])
# y_proba = y_probas.mean(axis=0) 